<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [1]:
# Install spark
!pip install pyspark
!pip install findspark

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Start session

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Search term analytics").getOrCreate()

23/06/20 19:36:30 WARN Utils: Your hostname, debian11host resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/06/20 19:36:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/20 19:36:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Download The search term dataset from the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2023-06-20 19:16:19--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv’

searchterms.csv     100%[===================>] 227.99K   251KB/s    in 0.9s    

2023-06-20 19:16:21 (251 KB/s) - ‘searchterms.csv’ saved [233457/233457]



In [8]:
# Load the csv into a spark dataframe
dataframe = spark.read.option("header", True).csv("searchterms.csv").cache()

In [9]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

number_of_rows = dataframe.count()
print(f"Dataframe rows count is: {number_of_rows}")

number_of_columns = len(dataframe.columns)
print(f"Dataframe columns count is: {number_of_columns}")

Dataframe rows count is: 10000
Dataframe columns count is: 4


In [10]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

dataframe.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [13]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

type(dataframe['searchterm'])

pyspark.sql.column.Column

In [14]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

dataframe.filter(dataframe['searchterm'] == 'gaming laptop').count()

499

In [15]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

top_5_most_frequent_terms = dataframe.groupby(['searchterm']) \
.agg({"searchterm": "count"}) \
.sort("count(searchterm)", ascending = False) \
.show(5)

+-------------+-----------------+
|   searchterm|count(searchterm)|
+-------------+-----------------+
|mobile 6 inch|             2312|
|    mobile 5g|             2301|
|mobile latest|             1327|
|       laptop|              935|
|  tablet wifi|              896|
+-------------+-----------------+
only showing top 5 rows



In [16]:
# The pretrained sales forecasting model is available at  the below url
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2023-06-20 19:49:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-06-20 19:49:50 (56.2 MB/s) - ‘model.tar.gz’ saved [1490/1490]



In [17]:
!tar -xf model.tar.gz

In [19]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

model = LinearRegressionModel.load('sales_prediction.model')

In [20]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[0,0,year,0]]
    columns = ["day", "month", "year", "search"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','year')
    predictions = model.transform(__)
    predictions.select('prediction').show()

predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/06/20 19:54:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
